In [1]:
import json
import os
from pprint import pprint
from tqdm import tqdm

PATH_TO_DATA = 'data/processed_for_chroma/reviews'
FILENAMES = os.listdir(PATH_TO_DATA)
review_data = dict()

for filename in FILENAMES:
    with open(f'{PATH_TO_DATA}/{filename}', 'r') as file:
        review_data[os.path.splitext(os.path.basename(filename))[0]] = json.load(file)

# This one is missing the doi key
del review_data['Earth_Science_Reviews']['metadatas'][292]
del review_data['Earth_Science_Reviews']['documents'][292]
del review_data['Earth_Science_Reviews']['ids'][292]

# postprocessing
for journal in review_data:
    for i, d in enumerate(review_data[journal]['metadatas']):
        # Convert stringified list to list
        d['reference'] = json.loads(d['reference'])
        d['doi'] = json.loads(d['doi'])

print(review_data.keys())
print(f"Journal keys: {review_data['Astro_Reviews'].keys()}")
paper = review_data['Astro_Reviews']['metadatas'][0]
print(paper.keys())
print(paper['doi'])
print(type(paper['doi']))
print(paper['reference'][:3])
all_reviews = [
    record for journal in review_data for record in review_data[journal]['metadatas']]
print(len(all_reviews))
print(all_reviews[0].keys())

dict_keys(['Earth_Science_Reviews', 'Planetary_Reviews', 'Astro_Reviews'])
Journal keys: dict_keys(['documents', 'metadatas', 'ids'])
dict_keys(['bibcode', 'abstract', 'aff', 'author', 'bibstem', 'doctype', 'doi', 'id', 'keyword', 'pubdate', 'title', 'read_count', 'reference', 'citation_count', 'citation', 'body'])
['10.1146/annurev.astro.46.060407.145222', '10.48550/arXiv.0909.0948']
<class 'list'>
['1929ApJ....70...11R', '1956RvMP...28...53S', '1958ZA.....46..108B']
2984
dict_keys(['bibcode', 'abstract', 'aff', 'author', 'bibstem', 'doctype', 'doi', 'id', 'keyword', 'pubdate', 'title', 'read_count', 'reference', 'citation_count', 'citation', 'body'])


In [2]:
PATH_TO_DATA = 'data/processed_for_chroma/research'
FILENAMES = os.listdir(PATH_TO_DATA)
research_data = dict()

for filename in FILENAMES:
    with open(f'{PATH_TO_DATA}/{filename}', 'r') as file:
        research_data[os.path.splitext(os.path.basename(filename))[
            0]] = json.load(file)


# postprocessing
for journal in research_data:
    for i, d in enumerate(research_data[journal]['metadatas']):
        if not 'doi' in d:
            del research_data[journal]['metadatas'][i]
            del research_data[journal]['documents'][i]
            del research_data[journal]['ids'][i]

for journal in research_data:
    for d in research_data[journal]['metadatas']:

        # Convert stringified list to list
        d['reference'] = json.loads(d['reference'])
        d['doi'] = json.loads(d['doi'])

print(research_data.keys())
research_data['Astro_Research'].keys()

all_data = all_reviews + \
    [record for journal in review_data for record in review_data[journal]['metadatas']]
print(f"All data: {len(all_data)}")

dict_keys(['Earth_Science_Research', 'Planetary_Research', 'Astro_Research'])
All data: 5968


In [3]:
# How many bibcodes are there in this paper's references that start with 1929 and end with 'R'?
import re
# pattern = r'^2000.*B$'
# matches = [s for s in paper['reference'] if re.match(pattern, s)]
# print(matches)

In [4]:

# Define the patterns
lastname = r"[A-Z][a-zA-ZÀ-ÖØ-öø-ÿ-]*(?:'[A-Z][a-zA-ZÀ-ÖØ-öø-ÿ-]*)?"
year = r"\(?(\d{4}[a-z]?)\)?"
name_sep = r",?\s"
INLINE_CITATION_PATTERN = fr"({lastname}(?:{name_sep}{lastname})*(?: et al.?)?)\s*{year}"

# Compile the regex pattern
inline_regex = re.compile(INLINE_CITATION_PATTERN)

print(inline_regex)

test = " Delbouille et al. 1981 the future (Section 5). 2. INGREDIENTS FOR SOLAR ABUNDANCE ANALYSIS 2.1. Observations Analyses of th"

# Find all matches using the compiled pattern
matches = inline_regex.finditer(test)
results = [match for match in matches]
print(f"Results: {len(results)}")

# Print the groups of each match
for i, result in enumerate(results):
    print(i+1, result.groups())

re.compile("([A-Z][a-zA-ZÀ-ÖØ-öø-ÿ-]*(?:'[A-Z][a-zA-ZÀ-ÖØ-öø-ÿ-]*)?(?:,?\\s[A-Z][a-zA-ZÀ-ÖØ-öø-ÿ-]*(?:'[A-Z][a-zA-ZÀ-ÖØ-öø-ÿ-]*)?)*(?: et al.?)?)\\s*\\(?(\\d{4}[a-z]?)\\)?")
Results: 1
1 ('Delbouille et al.', '1981')


### Get the inline citations from a paper's body

In [5]:
# def get_inline_citations(record: dict) -> list[tuple[str, str]]:
#     return [match.groups() for match in inline_regex.finditer(record['body'])]

# # inline_citations = [match.groups() for match in inline_regex.finditer(paper['body'])]
# inline_citations = get_inline_citations(paper)
# print(f"Results: {len(inline_citations)}")
# pprint(inline_citations[:5])

### Functions to resolve incline citation to bibcode

In [6]:
# for each result, get the first letter of the first author's last name
# get the year
def bibcode_regex(author: str, year: str):
    """
    Given first author and year, return a regex pattern for the
    corresponding bibcode
    """
    initial = author[0]
    year = year[:4] # cut off any letters at the end
    pattern = fr'^{year}.*{initial}$'
    return re.compile(pattern)

def bibcode_matches(inline_citation: tuple[str, str], references: list[str]) -> int:
    """
    Given an inline citation and a list of references, return the number of
    references that match the inline citation's bibcode regex pattern
    """
    pattern = bibcode_regex(*inline_citation)
    return [s for s in references if pattern.match(s)]

# def make_citation_bibcode_list(inline_citations: list[tuple[str, str]], references: list[str]) -> list[tuple[tuple[str, str], str]]:
#     """
#     Given a paper's list of inline citations and list of references, return a list of
#     tuples where the first element is the inline citation and the second element
#     is the corresponding bibcode from the references list where there is exactly one match
#     """
#     return [(citation, matches[0]) for citation in inline_citations 
#             if len((matches := bibcode_matches(citation, references))) == 1]

# usable_citations = make_citation_bibcode_list(inline_citations, paper['reference'])
# print(f"Results: {len(usable_citations)}")
# print(usable_citations[:5])

In [7]:
# def get_all_bibcodes_from_file(path: str) -> list[str]:
#     with open(path, 'r') as file:
#         return json.load(file)

# bibcodes = get_all_bibcodes_from_file('data/bibcodes.json')
# print(f"Results: {len(bibcodes)}")
# print(bibcodes[:5])

In [8]:
# def resolve_inline_references(records, bibcodes):
#     in_dataset, out_of_dataset = [], []
#     for record in tqdm(records, desc='Processing records'):
#         usable_citations = make_citation_bibcode_list(get_inline_citations(record), record['reference'])
#         for citation in usable_citations:
#             # Construct the citation dictionary
#             inline_citation, bibcode = citation
#             cite_dict = {'source_bibcode': record['bibcode'],
#                          'inline_citation': inline_citation, 
#                          'reference_bibcode': bibcode}

#             # Determine if the referenced bibcode is in the dataset or not
#             in_dataset.append(cite_dict) if bibcode in bibcodes else out_of_dataset.append(cite_dict)
#     return in_dataset, out_of_dataset

# have, dont_have = resolve_inline_references(review_data['Astro_Reviews']['metadatas'][:1], bibcodes)
# print(f"In dataset: {len(have)}")
# print(f"Out of dataset: {len(dont_have)}")
# pprint(dont_have[:2])

In [14]:
import pysbd

def get_inline_citations(text: str) -> list[tuple[str, str]]:
    return [match.groups() for match in inline_regex.finditer(text)]

def sentence_to_example(record, sentence):
    """
    Takes all the inline citations of a sentence and if it can resolve them to dois
    then it returns the """
    def citation_to_doi(citation):
        """
        Takes a single inline citation as tuple of (author, year) and determines if there is a unique
        matching bibcode in the record's references. If so, it continues to look for a unique
        doi matching that bibcode in the entire dataset. It returns the doi if resolved, otherwise None.
        """
        bibcodes = bibcode_matches(citation, record['reference'])
        if len(bibcodes) != 1:
            return None
        matching_dois = [record['doi'][0]
                         for record in all_data if record['bibcode'] == bibcodes[0]]
        if len(matching_dois) != 1:
            return None
        return matching_dois[0]
    
    inline_citations = get_inline_citations(sentence)
    citation_dois = []
    for citation in inline_citations:
        if not (doi := citation_to_doi(citation)):
            break
        citation_dois.append(doi)

    # If all citations resolved to dois, return the example
    if len(inline_citations) != len(citation_dois):
        return None
    return {
            'source_doi': record['doi'][0],
            'sentence': sentence,
            'citation_dois': citation_dois
           }


def create_examples_from_record(record):
    splitter = pysbd.Segmenter(language="en", clean=False)
    sentences = [s for s in splitter.segment(record['body']) if len(s) > 40]
    return [
        example for sentence in sentences if (example := sentence_to_example(record, sentence))
    ]
        

In [15]:
with open('data/test_set.jsonl', 'a') as file:
    for journal in review_data:
        for record in review_data[journal]['metadatas'][:10]:
            examples = create_examples_from_record(record)
            for example in examples:
                json.dump(example, file)
                file.write('\n')